## Autonomy equations for data-set from people in MSA  

There are 3 types of data in a data-set in Autonomy. They are confirmed cases from Taiwan CDC, symptom reports, and good behavior reports from Autonomy users in a MSA area. Follows are how we calculate these indicators


### Symptom Indicator
+ There are two types of symptoms: **Official** & **Customized** Symptom.
+ A user can adjust Weight for each symptom to have their score
+ The higher number symptoms reported, the lower the score
+ The score of symptoms is:

```Score = 100 * (1 -(SumOfTotalWeight/((TotalPeople*MaxWeightPerPerson)+SumOfCustomizedWeight)))```

+ Factors
	+ SumOfTotalWeight
		+ WeightMatrix : the weight that a user gives to each official symptom.
		```
		{
			Fever:   3,
			Cough:   2,
			Fatigue: 1,
			Breath:  1,
			Nasal:   1,
			Throat:  1,
			Chest:   2,
			Face:    2,
		}
		```
	+ TotalPeople : total people report in the MSA at specific period (ie. today) 
	+ MaxWeightPerPerson: the weights a person has, if the person reports all official symptoms
		+ use the WeightMatrix above, it is (3+2+1+1+1+1+2+2) = 13
	+ CustomizedWeight : the sum of total customized weights
	+ SumOfCustomizedWeight: CustomizedWeight for each symptom is 1 (so CustomizedWeight = CustomizedCount)
	+ Normalization: the score is 100 and the more symptoms and the lower the score
    
	 ```
		100 - 100*(WeightsReported/WeightWeExpectedAsA100Point)
		100*(1-(WeightsReported/WeightWeExpectedAsA100Point))
		100*(1-(SumOfTotalWeight/(((TotalPeople*MaxWeightPerPerson)+CustomizedWeight)))
	```

#### Example data-set of symptom report

In  a MSA area there are 5 reports from A,B,C and D

```
 A : Fever , customized01
 B : Fever, Dry Cought, Chest Pain
 C : Dry Cough, Fatique
 D : customized02
```


Data set will have below value
+ total count of customized symptoms
+ total people in the report
+ total count of each official symptom in the Data-set (we call it distribution)
+ weight metrix : user can define the weight for each official symptoms while customized weight always weight 1

In [1]:
totalCustomizedCountSymptom = 2
totalPeopleSymptom = 4

In [2]:

distributionSymptom = {
    'Fever':   2,
    'Cough':   2,
    'Fatigue': 1,
    'Breath':  0,
    'Nasal':   0,
    'Throat':  0,
    'Chest':   1,
    'Face':    0,
}


In [3]:

weightMatrixSymptom = {
    'Fever':   3,
    'Cough':   2,
    'Fatigue': 1,
    'Breath':  1,
    'Nasal':   1,
    'Throat':  1,
    'Chest':   2,
    'Face':    2,
}


#### Equation
+ calculate intermediate values for the final score

In [4]:
totalWeightSymptom = 0.0
for key, value  in distributionSymptom.items():
    totalWeightSymptom = totalWeightSymptom + value*weightMatrixSymptom[key]

print('totalWeightSymptom',totalWeightSymptom)
maxWeightPerPersonSymptom = 0.0
for key, value  in weightMatrixSymptom.items():
    maxWeightPerPersonSymptom = maxWeightPerPersonSymptom + value
print('maxWeightPerPersonSymptom:',maxWeightPerPersonSymptom)

totalWeightSymptom 13.0
maxWeightPerPersonSymptom: 13.0


#### Final Score

In [5]:
if ((totalWeightSymptom*maxWeightPerPersonSymptom)+totalCustomizedCountSymptom*1) > 0:
    scoreSymptom = 100*(1-(totalWeightSymptom/((totalPeopleSymptom*maxWeightPerPersonSymptom)+totalCustomizedCountSymptom*1)))
else:
    scoreSymptom = 100
print('**scoreSymptom', scoreSymptom)

**scoreSymptom 75.92592592592592


###  Behaviors Indicator

+ There are two types of behaviors: **Official** & **Customized** behavior.
+ The user can not change their **weight matrix** in Sprint3
+ Each Official behavior weight 1 , the same as customized behavior
+ The more good behaviors reported, the better the score
+ The score of symptoms is:

```Score = 100*(SumOfTotalWeight/((TotalPeople*MaxWeightPerPerson)+SumOfCustomizedWeight))```

#### Example data-set of behavior report

    ```
     A : CleanHand , customized01
	 B : SocialDistancing, TouchFace
	 C : CleanHand
	 D : customized02
	```


behavior data in the autonomy data-set will have values:
+ sum of total weights (official+customized good behaviors) 
+ total people in the report
+ a max weight per person can have
+ total customized weight 

#### Equation & final score


In [6]:
# SumOfTotalWeight = 2 + 2 + 1 + 1
SumOfTotalWeightBehavior = 6
TotalPeopleBehavior = 4 
# (We have 6 official behaviors)
MaxWeightPerPersonBehavior = 6 
SumOfCustomizedWeightBehavior = 2
if ((TotalPeopleBehavior*MaxWeightPerPersonBehavior)+SumOfCustomizedWeightBehavior) > 0:
    scoreBehavior = 100*(SumOfTotalWeightBehavior/((TotalPeopleBehavior*MaxWeightPerPersonBehavior)+SumOfCustomizedWeightBehavior))
else:
    scoreBehavior = 0
print('**scoreBehavior:', scoreBehavior)

**scoreBehavior: 23.076923076923077


## Confirmed cases indicator
+ The confirmed cases data is from [Corona Data Scraper CDS](https://coronadatascraper.com/#home) 
    + Taiwan use country level data
    + US use county level data
+ Score Equation　
    + Exponential Weight Average Method
   
$$1 - \dfrac{\displaystyle\sum_{i=1}^{14}e^{\frac{i}{2}}d_i}{\displaystyle\sum_{i=1}^{14}e^{\frac{i}{2}}(d_i+1)}$$


### Example 1 
In 14 days , each day the new confirm cases is 
 - day 1: 3
 - day 2: 2
 - day 3: 2
 - day 4: 1
 - day 14: 10 (last data reported)

$$1 - \dfrac{e^{0.5}*3 + e*2+ e^{1.5}*2 + e^2 * 1 + e^7*10 }{e^{0.5}*4 + e*3+ e^{1.5}*3 + e^2 * 2 + e^{2.5} +e^{3}+e^{3.5}+e^{4}+e^{4.5}+e^{5}+e^{5.5}+e^{6}+e^{6.5} + e^7*11} \\
\approx 0.20210651903685417
$$

The score is at about 20.2 since there are much cases in the nearest day.

### Example 2
In 14 days , each day the new confirm cases is 
 - day 1: 20
 - day 2: 2
 - day 3: 2
 - day 4: 1
 - day 13: 1
 - day 14: 1 (last data reported)
 
$$1 - \dfrac{e^{0.5}*20 + e*2+ e^{1.5}*2 + e^2 * 1 + e^{6.5}*2+ e^7*1 }{e^{0.5}*21 + e*3+ e^{1.5}*3 + e^2 * 2 + e^{2.5} +e^{3}+e^{3.5}+e^{4}+e^{4.5}+e^{5}+e^{5.5}+e^{6}+e^{6.5}*3+ e^7*2} \\
\approx 0.5287554499517347
$$

The score is at about 52.87 since lots of cases happended on the farthest day.

### A 14 days Examples

In [12]:
confirmsCases14Days = [3, 2, 2, 1, 0, 1, 30, 4, 0, 0, 3, 3, 4, 3]

#### Equation & final score

In [13]:
import math

numerator = 0
denominator = 0
day = 1
for cases in  confirmsCases14Days :
    power = day/2
    numerator = numerator +  math.exp(power)*cases
    denominator = denominator +  math.exp(power)*(cases+1)
    # print("numerator:",numerator, "denominator:",denominator, "day:",day)
    day += 1

scoreConfirm = 0
if denominator > 0 :
    scoreConfirm = 100*(1- numerator/denominator)

print('**ScoreConfirm:', scoreConfirm)

numerator: 4.946163812100385 denominator: 6.594885082800513 day: 1
numerator: 10.382727469018475 denominator: 14.749730568177648 day: 2
numerator: 19.346105609694604 denominator: 28.194797779191845 day: 3
numerator: 26.735161708625256 denominator: 42.97290997705315 day: 4
numerator: 26.735161708625256 denominator: 55.155403937756624 day: 5
numerator: 46.820698631812924 denominator: 95.32647778413195 day: 6
numerator: 1040.2842573925823 denominator: 1121.9054885035935 day: 7
numerator: 1258.6768575251592 denominator: 1394.8962386693147 day: 8
numerator: 1258.6768575251592 denominator: 1484.9133699698366 day: 9
numerator: 1258.6768575251592 denominator: 1633.3265290724132 day: 10
numerator: 1992.7526543178203 denominator: 2612.0942581292948 day: 11
numerator: 3203.0390347960256 denominator: 4225.809432100235 day: 12
numerator: 5863.605566973472 denominator: 7551.517597322045 day: 13
numerator: 9153.505042258848 denominator: 11938.050231035879 day: 14
**ScoreConfirm: 23.324957885818954


##  Overall score
Assume we can have normalized score ([0-100]) from each indicator above, we want to come up with an overall score for users as the final indicator. The most straighfoward way could be a linear combination:

`final socre = c1*symptom_score + c2*behavior_score + c3*confirm_case_score`

### Example
Coefficients for each score is
    
    coefficient1 = 0.25 (symptom)
    coefficient2 = 0.25 (behavior)
    coefficient13 = 0.50 (confirmed cases)
    
We use scoreSymptom , scoreBehavior and scoreConfirm from Above

In [9]:
coefSymptom=0.25
coefBehavior=0.25
coefConfirm=0.5
score= coefSymptom*scoreSymptom+ coefBehavior*scoreBehavior + coefConfirm*scoreConfirm
print('**Autonomy Overall Score:',score)

**Autonomy Overall Score: 28.175481947863293
